In [1]:
import seaborn as sns

In [4]:
import pandas as pd

In [5]:
df=pd.read_csv('tips.csv')

In [6]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [7]:
df['day'].uniqueque()

array(['Sun', 'Sat', 'Thur', 'Fri'], dtype=object)

In [8]:
df['time'].unique()

array(['Dinner', 'Lunch'], dtype=object)

In [9]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [10]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [11]:
# convert the time into labelEncoder

In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
enc=LabelEncoder()

In [14]:
df['time']=enc.fit_transform(df['time'])

In [15]:
df['time'].unique()

array([0, 1])

In [16]:
X=df.drop(labels='time',axis=1)

In [17]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [20]:
y=df.time

In [21]:
y

0      0
1      0
2      0
3      0
4      0
      ..
239    0
240    0
241    0
242    0
243    0
Name: time, Length: 244, dtype: int32

In [22]:
X['day'].value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=42)

In [38]:
from sklearn.pipeline import Pipeline
# to create pipeline for automate the feature engennering automation
# to handle missing values automatically 
from sklearn.impute import SimpleImputer

In [28]:
from sklearn.preprocessing import StandardScaler
# feature scalling

In [29]:
from sklearn.preprocessing import OneHotEncoder
# categorical values to binary class

In [31]:
from sklearn.compose import ColumnTransformer
# after we apply multiple transoform to the pipeline to use in model training

In [32]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [33]:
cat_cols=['sex','smoker','day']
num_cols=['total_bill','tip','size']

In [34]:
# numerical pipeline 
num_pipeline=Pipeline(steps=[
    ('impute',SimpleImputer(strategy='median')),
    ('scale',StandardScaler())
])

In [35]:
# category pipeline
cat_pipeline=Pipeline(steps=[
    ('impute',SimpleImputer(strategy='most_frequent')),
    ('onehotencode',OneHotEncoder())
])

In [39]:
preprocess=ColumnTransformer([
    ('num_pipeline',num_pipeline,num_cols),
    ('cat_pipeline',cat_pipeline,cat_cols)
])

In [40]:
preprocess

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='median')),
                                                 ('scale', StandardScaler())]),
                                 ['total_bill', 'tip', 'size']),
                                ('cat_pipeline',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencode',
                                                  OneHotEncoder())]),
                                 ['sex', 'smoker', 'day'])])

In [42]:
X_train=preprocess.fit_transform(X_train)

In [43]:
X_test=preprocess.transform(X_test)

In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [51]:
models={
    'Random_forest':RandomForestClassifier(),
    'Logistic_regression':LogisticRegression(),
    'Dtree':DecisionTreeClassifier()
}

In [ ]:
models.values

In [52]:
from sklearn.metrics import accuracy_score

In [59]:
def evalute_model(X_train,y_train,X_test,y_test,models):
    reports={}
    for i in range(len(models)):
        model=list(models.values())[i]
        model.fit(X_train,y_train)
        
        y_pred=model.predict(X_test)
        test_acc=accuracy_score(y_test,y_pred)
        # get the accuracy of the model
        reports[list(models.keys())[i]] =  test_acc
        
    return reports
        

In [60]:
evalute_model(X_train,y_train,X_test,y_test,models)

{'Random_forest': 0.9753086419753086,
 'Logistic_regression': 0.9876543209876543,
 'Dtree': 0.9506172839506173}

In [61]:
classifier=RandomForestClassifier()

In [68]:
params={
    'max_depth':[3,5,10,None],
    'n_estimators':[100,200,300],
    'criterion':['gini','entropy']
}

In [69]:
from sklearn.model_selection import RandomizedSearchCV

In [70]:
cv=RandomizedSearchCV(classifier,param_distributions=params,cv=5,verbose=3,scoring='accuracy')

In [71]:
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.939 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.970 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.909 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.906 total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=10, n_estimators=100;, score=0.906 total time=   0.3s
[CV 1/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.939 total time=   0.7s
[CV 2/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.939 total time=   0.7s
[CV 3/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.909 total time=   0.7s
[CV 4/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.938 total time=   0.7s
[CV 5/5] END criterion=entropy, max_depth=None, n_estimators=300;, score=0.906 tot

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [72]:
cv.best_params_

{'n_estimators': 300, 'max_depth': 3, 'criterion': 'gini'}

In [73]:
# internal assignment total bill is o/p and all the params are i/p feature
# do it with random forest